In [1]:
!pip install pyspark

In [70]:
import pyspark
import pandas as pd
from pyspark.sql.types import *

In [71]:
import numpy as np
from pyspark.sql import SparkSession


In [72]:
import sys
import math

In [2]:
!pip install findspark
import findspark
findspark.init()
findspark.find()

'c:\\Users\\komfl\\anaconda3\\Lib\\site-packages\\pyspark'

In [73]:
spark=SparkSession.builder.appName('firstapp').getOrCreate()

In [74]:
#load the dataset and add an index for each rows
data = spark.read.csv('./test.csv', header=True, inferSchema=True)
data = data.drop('Name')
df = data.toPandas()
df_index = pd.DataFrame({'index': range(0, len(df))})
result_df = pd.concat([df, df_index], axis=1)
data = spark.createDataFrame(result_df)
# asf = spark.createDataFrame(ee)
# asf.show()

+---+---+---+---+-----+
|Age|  X|  Y|  Z|index|
+---+---+---+---+-----+
| 25|  5|  2|  3|    0|
| 50| 15|  4|  9|    1|
| 75| 30|  6| 15|    2|
|100| 45|  8| 21|    3|
|125| 60| 10| 27|    4|
|150| 75| 12| 33|    5|
|175| 90| 14| 39|    6|
|200|105| 16| 45|    7|
|225|120| 18| 51|    8|
|250|135| 20| 57|    9|
|275|150| 22| 63|   10|
|300|165| 24| 69|   11|
|325|180| 26| 75|   12|
|350|195| 28| 81|   13|
|375|210| 30| 87|   14|
|400|225| 32| 93|   15|
|425|240| 34| 99|   16|
|450|255| 36|105|   17|
|475|270| 38|111|   18|
|500|285| 40|117|   19|
+---+---+---+---+-----+
only showing top 20 rows



In [118]:
#Select randomly five initials centroids
centers = data.toPandas().sample(n=5)
centers
# npCenters = spark.createDataFrame(interDf)
# centers = npCenters

,Age,X,Y,Z,index
11,300,165,24,69,11
25,650,375,52,153,25
13,350,195,28,81,13
23,600,345,48,141,23
4,125,60,10,27,4


In [132]:
for g in data.collect():
    a = g.asDict()
    del a['index']
    print(a)
    l = Row(**a)
    print(l)
    print(g)

{'Age': 25, 'X': 5, 'Y': 2, 'Z': 3}
Row(Age=25, X=5, Y=2, Z=3)
Row(Age=25, X=5, Y=2, Z=3, index=0)
{'Age': 50, 'X': 15, 'Y': 4, 'Z': 9}
Row(Age=50, X=15, Y=4, Z=9)
Row(Age=50, X=15, Y=4, Z=9, index=1)
{'Age': 75, 'X': 30, 'Y': 6, 'Z': 15}
Row(Age=75, X=30, Y=6, Z=15)
Row(Age=75, X=30, Y=6, Z=15, index=2)
{'Age': 100, 'X': 45, 'Y': 8, 'Z': 21}
Row(Age=100, X=45, Y=8, Z=21)
Row(Age=100, X=45, Y=8, Z=21, index=3)
{'Age': 125, 'X': 60, 'Y': 10, 'Z': 27}
Row(Age=125, X=60, Y=10, Z=27)
Row(Age=125, X=60, Y=10, Z=27, index=4)
{'Age': 150, 'X': 75, 'Y': 12, 'Z': 33}
Row(Age=150, X=75, Y=12, Z=33)
Row(Age=150, X=75, Y=12, Z=33, index=5)
{'Age': 175, 'X': 90, 'Y': 14, 'Z': 39}
Row(Age=175, X=90, Y=14, Z=39)
Row(Age=175, X=90, Y=14, Z=39, index=6)
{'Age': 200, 'X': 105, 'Y': 16, 'Z': 45}
Row(Age=200, X=105, Y=16, Z=45)
Row(Age=200, X=105, Y=16, Z=45, index=7)
{'Age': 225, 'X': 120, 'Y': 18, 'Z': 51}
Row(Age=225, X=120, Y=18, Z=51)
Row(Age=225, X=120, Y=18, Z=51, index=8)
{'Age': 250, 'X': 135, 'Y

In [141]:
#Map operation
cols = [item for item in data.columns if(item != 'index')]
def map(x):
    minDist = sys.float_info.max
    index = -1
    for i, center in centers.iterrows():
        distCenter = 0
        for column in cols:
            distCenter = distCenter + (x[column] - center[column])**2
        distCenter = math.sqrt(distCenter)
        if(minDist > distCenter):
             minDist = distCenter
             index = i
        y = x.asDict()
        # del y['index']
        y['num'] = 1
        toSave = Row(**y)         
    return (index, toSave)
rdd = data.rdd.map(lambda x: map(x)).groupByKey()
e = rdd.toDF()
for f in e.collect():
    print(f)


Row(_1=23, _2=Row(data=[Row(Age=500, X=285, Y=40, Z=117, num=1), Row(Age=525, X=300, Y=42, Z=123, num=1), Row(Age=550, X=315, Y=44, Z=129, num=1), Row(Age=575, X=330, Y=46, Z=135, num=1), Row(Age=600, X=345, Y=48, Z=141, num=1)], index=0, maxindex=5))
Row(_1=4, _2=Row(data=[Row(Age=25, X=5, Y=2, Z=3, num=1), Row(Age=50, X=15, Y=4, Z=9, num=1), Row(Age=75, X=30, Y=6, Z=15, num=1), Row(Age=100, X=45, Y=8, Z=21, num=1), Row(Age=125, X=60, Y=10, Z=27, num=1), Row(Age=150, X=75, Y=12, Z=33, num=1), Row(Age=175, X=90, Y=14, Z=39, num=1), Row(Age=200, X=105, Y=16, Z=45, num=1)], index=0, maxindex=8))
Row(_1=25, _2=Row(data=[Row(Age=625, X=360, Y=50, Z=147, num=1), Row(Age=650, X=375, Y=52, Z=153, num=1), Row(Age=675, X=390, Y=54, Z=159, num=1), Row(Age=700, X=405, Y=56, Z=165, num=1), Row(Age=725, X=420, Y=58, Z=171, num=1)], index=0, maxindex=5))
Row(_1=11, _2=Row(data=[Row(Age=225, X=120, Y=18, Z=51, num=1), Row(Age=250, X=135, Y=20, Z=57, num=1), Row(Age=275, X=150, Y=22, Z=63, num=1), Row

In [145]:
def reduce(a, b):
    c = {}
    for column in cols:
        c[column] = float(a[column]) + float(b[column])
    c['num'] = a['num'] + b['num']
    toSave = Row(**c)
    return toSave
rdd = data.rdd.map(lambda x: map(x)).reduceByKey(lambda a,b: reduce(a, b))
e = rdd.toDF()
for f in e.collect():
    g = f.asDict()
    for col in cols:
        g[col] = g[col]/g['num']
    print(g)


Row(_1=23, _2=Row(Age=2750.0, X=1575.0, Y=220.0, Z=645.0, num=5))
Row(_1=4, _2=Row(Age=900.0, X=425.0, Y=72.0, Z=192.0, num=8))
Row(_1=25, _2=Row(Age=3375.0, X=1950.0, Y=270.0, Z=795.0, num=5))
Row(_1=11, _2=Row(Age=1375.0, X=750.0, Y=110.0, Z=315.0, num=5))
Row(_1=13, _2=Row(Age=2475.0, X=1395.0, Y=198.0, Z=576.0, num=6))


In [147]:
for f in e.collect():
    g = f['_2'].asDict()
    for col in cols:
        g[col] = g[col]/g['num']
    print(g)


{'Age': 550.0, 'X': 315.0, 'Y': 44.0, 'Z': 129.0, 'num': 5}
{'Age': 112.5, 'X': 53.125, 'Y': 9.0, 'Z': 24.0, 'num': 8}
{'Age': 675.0, 'X': 390.0, 'Y': 54.0, 'Z': 159.0, 'num': 5}
{'Age': 275.0, 'X': 150.0, 'Y': 22.0, 'Z': 63.0, 'num': 5}
{'Age': 412.5, 'X': 232.5, 'Y': 33.0, 'Z': 96.0, 'num': 6}


In [148]:
centers

,Age,X,Y,Z,index
11,300,165,24,69,11
25,650,375,52,153,25
13,350,195,28,81,13
23,600,345,48,141,23
4,125,60,10,27,4
